# DTree Example

In [25]:
from DTree import DTree
from DTree.Formula import formula

from Lifting import Lifting
import json
import time

In [26]:
Lifting = Lifting.Lifting()
with open("data/hard/factorisedFormulas/17e_n_name4_factorised.json") as fp:
    JsonObject = json.load(fp)
lineage = JsonObject[0]  
testFormula = formula(lineage)
current_id = Lifting.lift_read_once_clause(testFormula)
testFormula.reduce_depth(testFormula)
Lifting.lift_formula(testFormula, current_id)
testFormula.reduce_depth(testFormula)

In [27]:
dtree = DTree.Dtree(testFormula, None)
print(dtree)

DTREE_GATE.Independent_And DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Independent_And {"operator": and,
 "subformula": [198, d8d9e884-f059-4224-aafb-ebce8e1629f4]} {"operator": and,
 "subformula": [199, 194, 195, 196, 197]} DTREE_GATE.Independent_Or {"operator": and,
 "subformula": [444b137e-a3e2-4433-8caf-68c61c62c5d9, 93]} {"operator": or,
 "subformula": [65, 66, 67, 68, 94, 95, 96]} DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Exclusive_Or True DTREE_GATE.Exclusive_Or True DTREE_GATE.Exclusive_Or True False DTREE_GATE.Independent_Or DTREE_GATE.Exclusive_Or True DTREE_GATE.Exclusive_Or True DTREE_GATE.Exclusive_Or True False {"operator": or,
 "subformula": [143, 144]} {"operator": or,
 "subformula": [17, 18, 20, 171, 172, 173]} DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Independent_Or 162 {"operator": or,
 "subformula": [269, 80, 202]} DTREE_GATE.Exclu

In [28]:
def calculate_banzhaf2(dtree, timeout=3600):
    if dtree.variable_count == 0:
        return {"status": "Success", "expansion_time": 0, "calculation_time": 0, "depth": 0, "banzhaf_values": {}}
    banzhaf_vals = {}
    start = time.time()
    for fact in dtree.variables:
        banzhaf_vals[fact] = dtree.critical_assignments_fact(fact)
        mid = time.time()
        if mid - start > timeout:
            return {"status": "Failed", "calculation_time": "Failed", "banzhaf_values": banzhaf_vals}
    end = time.time()
    calc_time = end - start
    return {"status": "Success","calculation_time": calc_time, "banzhaf_values": banzhaf_vals}

In [29]:
result = calculate_banzhaf2(dtree, 3600)
print(result)

{'status': 'Success', 'calculation_time': 0.012000083923339844, 'banzhaf_values': {'209': 24997410, '127': 24997410, '133': 24997410, '197': 196830, '108': 24997410, '205': 24997410, '91': 24997410, '264': 24997410, '8': 24997410, '225': 24997410, '270': 24997410, '217': 24997410, '134': 24997410, '219': 8332470, '40': 24997410, '263': 24997410, '7': 24997410, '110': 24997410, '35': 24997410, '152': 24997410, '156': 24997410, '65': 24997410, '175': 24997410, '116': 24997410, '885a829b-c4b0-4d16-b9c8-1daaca6f4316': 24997410, '164': 24997410, '176': 24997410, '24': 24997410, '268': 24997410, '269': -8332470, '214': 24997410, '234': 24997410, '201': 24997410, '277': 24997410, '246': 24997410, '344ea546-9f57-48b5-9ecc-f2b8c799fd8e': 0, '163': 24997410, '64': 24997410, '102': 24997410, '200': 24997410, '203': 8332470, '56': 24997410, '73c18799-b803-417e-95be-6008632605b5': 0, '106': 24997410, '139': 24997410, '178': 24997410, '15': 24997410, '273': 24997410, '39': 24997410, '183': 24997410,